In [1]:
import folium

map_osm=folium.Map(location=[55.993837, 40.017943], zoom_start=15)
folium.Marker([55.9956, 40.0046], popup='My little house', icon=folium.Icon(color='green', icon='home')).add_to(map_osm)
map_osm.add_child(folium.LatLngPopup())

map_osm

In [12]:
import pandas as pd
import vacija as va

lst_work_fileds=['year', 'latitude', 'longtitude', 'wall_type', 'floors', 'gutter_type', 'address',
                  'entrance', 'gas_supply_type', 'living_rooms', 'square', 'living_square', 'warming_type',
                'cold_water_type', 'hot_water_type']
dtfHouses=pd.read_csv(r'/home/egor/data/Houses/vlad_obl_houses.csv', encoding='cp1251', sep=';', index_col='id')
df4map=va.clean_for_research1(dtfHouses)[lst_work_fileds]
#df4map=df4map.ix[df4map['address'].str.contains('Собинка|Лакинск')]
#df4map
df4map.sort_values('year', inplace=True)
#df4map=df4map.drop(df4map['latitude'].isnull())
df4map_year_marker=df4map[df4map['latitude'].notnull()]

#дубликаты по координатам!!!
df4map_year_marker[['latitude', 'longtitude']]=df4map_year_marker[['latitude', 'longtitude']].round(5)
dtf=df4map_year_marker[df4map_year_marker.duplicated(['latitude', 'longtitude'])]
print(dtf.shape)
df4map_year_marker=df4map_year_marker.drop_duplicates(subset=['latitude', 'longtitude'])

dct_walls_colors={'Каменные, кирпичные': 'maroon', 
 'Иные': 'lightblue', 
 'Деревянные': 'burlywood', 
 'Смешанные': 'peru', 
 'Блочные': 'lightgray', 
 'Панельные': 'lightgreen', 
 'Монолитные':'plum'}

#print(df4map_year_marker)
print(dtfHouses.columns.tolist())
print(dtfHouses['warming_type'].unique().tolist())
print(dtfHouses['cold_water_type'].unique().tolist())

/home/egor/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(3478, 15)
['address', 'year', 'base_type', 'cesspools_volume', 'cold_water_type', 'common_hold_square', 'electo_type', 'electro_inputs', 'elevators', 'energo_class', 'entrance', 'ext_info', 'face_type', 'fire_exting_type', 'floors', 'floors_min', 'garbage_chute_count', 'garbage_chute_type', 'gas_supply_type', 'gutter_type', 'hot_water_type', 'kap_rem_fund', 'land_plot_area', 'latitude', 'living_rooms', 'living_square', 'longtitude', 'manager', 'not_living_rooms', 'not_living_square', 'other', 'overlap_type', 'parking_area', 'playground', 'roof_type', 'roofing_type', 'rooms', 'seria', 'sportground', 'square', 'type', 'underground_square', 'vent_type', 'wall_type', 'warming_type', 'water_disposal_type', 'wreck']
['Не заполнено', 'Центральное', 'Квартирное отопление (квартирный котел)', 'Автономная котельная (крышная, встроенно-пристроенная)', 'Отсутствует', 'Электроотопление', 'Печное']
['Не заполнено', 'Отсутствует', 'Центральное', 'Автономное']


/home/egor/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2440: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [16]:
from folium import IFrame, FeatureGroup, LayerControl
import numpy as np

lst_years=list(range(1870, 2030, 20))
ci_start_year=1890
y_pairs=[(lst_years[i], lst_years[i+1]) for i in range(len(lst_years)-1)]
y_pairs[0]=(0, ci_start_year)

mrk_colors=['black', 'gray', 'lightgray', 'darkred', 'red', 'lightred', 'darkgreen', 'green', 'lightgreen', 'darkblue',
           'blue', 'lightblue', 'darkpurple', 'purple', 'pink', 'beige']

#print(list(zip(y_pairs,mrk_colors)))

def make_popup(df, year=True):
    tbl_h='''<font size='xx-small'><table width='100%' border='0'>'''
    tbl_mess='''<tr>
                <td width='40%' style='font-size:small'><b>{0}:</b></td>
                <td width='60%' style='font-size:small'>{1}</td>
                </tr>'''
    tbl_end='''</table></font>'''
    tbl_mm='''
                <tr>
                <td colspan=2 style='font-size:small'><b>{0}</b></td>
                </tr>'''
    tbl_body=''
    tbl_body+= tbl_mm.format(df['address'])
    if year:
        tbl_body+= tbl_mess.format('Год постройки', round(df['year']))
    else:
        tbl_body+= tbl_mm.format('Год постройки отсутствует')
    tbl_body+= tbl_mess.format('Материал стен', df['wall_type'])
    tbl_body+= tbl_mess.format('Этажность', df['floors'])
    tbl_body+= tbl_mess.format('Подъездов', df['entrance'])
    tbl_body+= tbl_mess.format('Газоснабжение', df['gas_supply_type'])
    tbl_body+= tbl_mess.format('Отопление', df['warming_type'])
    tbl_body+= tbl_mess.format('Вода хол.', df['cold_water_type'])
    tbl_body+= tbl_mess.format('Вода гор.', df['hot_water_type'])
    tbl_body+= tbl_mess.format('Жилая площадь', df['living_square'])
    tbl_body+= tbl_mess.format('Общая площадь', df['square'])
    return tbl_h+tbl_body+tbl_end



In [25]:
mp_vlad=mapa=folium.Map(location=[56.13655, 40.39658], zoom_start=8)#folium.Map(location=[56.13655, 40.39658], zoom_start=8)
d_clusters={}

def draw_marker(dtfrm, add_to=None, num_sides=3, fl_color='black', sd_color='grey', rad=7, is_year=True):
    for ir in dtfrm.iterrows():
        #print('\t{0} - {1} (lat:{2} - long:{3})'.format(ir[0], ir[1]['address'], 
        #                                                ir[1]['latitude'], ir[1]['longtitude']))
        coord=[ir[1]['latitude'], ir[1]['longtitude']]
        iframe=folium.IFrame(make_popup(ir[1], year=is_year), 270, 170)
        folium.RegularPolygonMarker(coord, popup=folium.Popup(iframe, max_width=2650), 
                                    fill_color=fl_color, color=sd_color, weight=1,
                                    number_of_sides=num_sides, 
                                    radius=rad).add_to(add_to)

#clstr=folium.MarkerCluster().add_to(mp_vlad)
ff_groups={k:FeatureGroup(name=k) for k in dct_walls_colors.keys()}
fcl_clstr={k:folium.MarkerCluster().add_to(ff_groups[k]) for k in dct_walls_colors.keys()}

for n_o_s, t in enumerate(y_pairs):
    for wt in dct_walls_colors.keys():
        m1=df4map_year_marker['wall_type']==wt
        m2=df4map_year_marker['year'].between(t[0], t[1], inclusive=False)
        dtf=df4map_year_marker.ix[m2 & m1]
        #print(n_o_s, t, wt, dtf.shape)
        draw_marker(dtf, add_to=fcl_clstr[wt], num_sides=n_o_s+3, fl_color=dct_walls_colors[wt])
        
for wt in dct_walls_colors.keys():
    m1=df4map_year_marker['wall_type']==wt
    m0=df4map_year_marker['year'].isnull()
    dtf0=df4map_year_marker.ix[m0 & m1]
    draw_marker(dtf0, add_to=fcl_clstr[wt], num_sides=3, fl_color=dct_walls_colors[wt], is_year=False)
    #draw_marker(dtf0, add_to=ff_groups[wt], num_sides=3, fl_color=dct_walls_colors[wt], is_year=False)


for ff in ff_groups.values():
    ff.add_to(mp_vlad)
LayerControl().add_to(mp_vlad)

#print(d_clusters)
mp_vlad.save('vladimir_obl.html')

In [24]:
#df4map=df4map.ix[df4map['address'].str.contains('Собинка|Лакинск')]
df_detail=df4map_year_marker.ix[df4map_year_marker['address'].str.contains('Собинка|Лакинск')]

mapa=folium.Map(location=[55.993837, 40.017943], zoom_start=13)

ff_groups={k:FeatureGroup(name=k) for k in dct_walls_colors.keys()}
fcl_clstr={k:folium.MarkerCluster().add_to(ff_groups[k]) for k in dct_walls_colors.keys()}

for n_o_s, t in enumerate(y_pairs):
    for wt in dct_walls_colors.keys():
        m1=df_detail['wall_type']==wt
        m2=df_detail['year'].between(t[0], t[1], inclusive=False)
        dtf=df_detail.ix[m2 & m1]
        #print(n_o_s, t, wt, dtf.shape)
        #draw_marker(dtf, add_to=mapa, num_sides=n_o_s+3, fl_color=dct_walls_colors[wt])
        draw_marker(dtf, add_to=fcl_clstr[wt], num_sides=n_o_s+3, fl_color=dct_walls_colors[wt])
        
for wt in dct_walls_colors.keys():
    m1=df_detail['wall_type']==wt
    m0=df_detail['year'].isnull()
    dtf0=df_detail.ix[m0 & m1]
    draw_marker(dtf0, add_to=fcl_clstr[wt], num_sides=3, fl_color=dct_walls_colors[wt], is_year=False)
        
for ff in ff_groups.values():
    ff.add_to(mapa)
LayerControl().add_to(mapa)
mapa
#mapa.save('vladimir_obl.html')

In [73]:
dtf=df4map_year_marker[df4map_year_marker.duplicated(['latitude', 'longtitude'])]

print(dtf.shape)


(0, 12)
